In [1]:
import pickle
import numpy as np

In [2]:
final_files_save_path = '1_clustering_single_level_weighted/pkls/clustering_output/jan/7/16_37_39_15000_0.8_0.05_0.05_0.05_0.05'
# final_files_save_path = 'pkls/clustering_output/nov/11/6_00_500_w2_0.2_0.2_0.3_0.15_0.15'

In [5]:
with open(f'{final_files_save_path}/frame_to_cluster_mapping.pkl', 'rb') as f:
    frame_to_cluster_mapping = pickle.load(f)

In [6]:
frame_to_cluster_mapping_transformed = {}
for key in frame_to_cluster_mapping:
    frame_to_cluster_mapping_transformed[key] = [cluster_id for _, cluster_id in frame_to_cluster_mapping[key]]

In [10]:
for key in frame_to_cluster_mapping_transformed:
    print(key, frame_to_cluster_mapping_transformed[key])
    break

01-01-01-01-01-01-01 [5582, 5582, 13043, 13043, 13043, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 3108, 3108, 3108, 3108, 3108, 3108, 3108, 13619, 5582, 5582, 5582, 13619, 264, 264, 3108, 14973, 14973, 14973, 14973, 14973, 14973, 14973, 14973, 975, 975, 975, 10420, 10420, 11027, 4390, 11027, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 10420, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401, 13401]


# Overall video statistics

In [31]:
# Calculate unique clusters and average run length per video
video_stats = {}
total_unique_clusters = 0
total_avg_run_length = 0
num_videos = len(frame_to_cluster_mapping_transformed)

all_unique_clusters = []
all_run_lengths = []
all_video_lengths = []
all_cluster_changes = []

for video_name, cluster_ids in frame_to_cluster_mapping_transformed.items():
    # Get unique clusters
    unique_clusters = len(set(cluster_ids))
    total_unique_clusters += unique_clusters
    all_unique_clusters.append(unique_clusters)
    
    # Calculate run lengths
    current_cluster = cluster_ids[0]
    current_run = 1
    runs = []
    cluster_changes = 0
    
    for cluster_id in cluster_ids[1:]:
        if cluster_id == current_cluster:
            current_run += 1
        else:
            runs.append(current_run)
            current_run = 1
            current_cluster = cluster_id
            cluster_changes += 1
    runs.append(current_run)  # Add the last run
    
    avg_run_length = sum(runs) / len(runs)
    total_avg_run_length += avg_run_length
    
    all_run_lengths.extend(runs)
    all_video_lengths.append(len(cluster_ids))
    all_cluster_changes.append(cluster_changes)

# Calculate overall statistics
overall_avg_unique_clusters = total_unique_clusters / num_videos
overall_avg_run_length = total_avg_run_length / num_videos

# Additional statistics
min_unique_clusters = min(all_unique_clusters)
max_unique_clusters = max(all_unique_clusters)
std_unique_clusters = np.std(all_unique_clusters)

min_run_length = min(all_run_lengths)
max_run_length = max(all_run_lengths)
std_run_length = np.std(all_run_lengths)

avg_video_length = np.mean(all_video_lengths)
std_video_length = np.std(all_video_lengths)

avg_cluster_changes = np.mean(all_cluster_changes)
std_cluster_changes = np.std(all_cluster_changes)

# Print comprehensive results
print("\nOverall Statistics:")
print("-" * 50)
print("\nUnique Clusters per Video:")
print(f"Average: {overall_avg_unique_clusters:.2f}")
print(f"Minimum: {min_unique_clusters}")
print(f"Maximum: {max_unique_clusters}")
print(f"Standard Deviation: {std_unique_clusters:.2f}")

print("\nRun Lengths (cluster_ids):")
print(f"Average across all videos: {overall_avg_run_length:.2f}")
print(f"Minimum: {min_run_length}")
print(f"Maximum: {max_run_length}")
print(f"Standard Deviation: {std_run_length:.2f}")

print("\nVideo Lengths in terms of frames")
print(f"Average: {avg_video_length:.2f} frames")
print(f"Standard Deviation: {std_video_length:.2f} frames")

print("\nCluster Changes:")
print(f"Average Cluster unique transitions per video: {avg_cluster_changes:.2f}")
print(f"Standard Deviation: {std_cluster_changes:.2f}")



Overall Statistics:
--------------------------------------------------

Unique Clusters per Video:
Average: 12.63
Minimum: 1
Maximum: 53
Standard Deviation: 7.47

Run Lengths (cluster_ids):
Average across all videos: 4.92
Minimum: 1
Maximum: 116
Standard Deviation: 4.88

Video Lengths in terms of frames
Average: 124.56 frames
Standard Deviation: 43.40 frames

Cluster Changes:
Average Cluster unique transitions per video: 31.28
Standard Deviation: 18.82


# Check if a cluster has frames from the same video

In [29]:
cluster_to_video_mapping = {}

for video, clusters in frame_to_cluster_mapping_transformed.items():
    for cluster_id in clusters:
        if cluster_id not in cluster_to_video_mapping:
            cluster_to_video_mapping[cluster_id] = set()
        cluster_to_video_mapping[cluster_id].add(video)

# Step 2: Check if any cluster is associated with only one video
single_video_clusters = [
    cluster_id for cluster_id, videos in cluster_to_video_mapping.items() if len(videos) == 1
]

# # Output the result
# if single_video_clusters:
#     print(f"There are clusters that only contain frames from one video: {single_video_clusters}")
# else:
#     print("No clusters contain frames from only one video.")

print("The number of clusters that have frames from a single video", len(single_video_clusters))

151


# Check if a cluster has frames from the same actor

In [28]:
cluster_to_actor_mapping = {}

for video, clusters in frame_to_cluster_mapping_transformed.items():
    for cluster_id in clusters:
        if cluster_id not in cluster_to_actor_mapping:
            cluster_to_actor_mapping[cluster_id] = set()
        if video[0] == "M":
            cluster_to_actor_mapping[cluster_id].add(video.split("/")[0])
        else:
            cluster_to_actor_mapping[cluster_id].add(video.split("-")[-1])

# Step 2: Check if any cluster is associated with only one video
single_actor_clusters = [
    cluster_id for cluster_id, actor in cluster_to_actor_mapping.items() if len(actor) == 1
]

# # Output the result
# if single_video_clusters:
#     print(f"There are clusters that only contain frames from one video: {single_video_clusters}")
# else:
#     print("No clusters contain frames from only one video.")

print("The number of clusters that only have data from a single actor are:", len(single_actor_clusters))

 14380
